# AmsterdamUMCdb Login & Connection Setup
This notebook provides the basic setup to connect to the AmsterdamUMCdb on Google BigQuery.
Fill in the form fields on the right and run the cells.

## 1. Project Configuration
Enter your Google Cloud Project ID and the dataset details.

In [ ]:
#@title Project Configuration { display-mode: "form" }

PROJECT_ID = "datathon-484308" #@param {type:"string"}
DATASET_PROJECT_ID = 'amsterdamumcdb' #@param {type:"string"}
DATASET_ID = 'van_gogh_2026_datathon' #@param {type:"string"}
LOCATION = 'eu' #@param {type:"string"}

import os
from google.colab import auth
from google.cloud import bigquery

# Set environment variable for Google Cloud Project
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

print(f"Project ID set to: {PROJECT_ID}")
print(f"Dataset: {DATASET_PROJECT_ID}.{DATASET_ID} (Location: {LOCATION})")

## 2. Authentication
Run this cell to authenticate your Google account. You will need to use the account that has access to AmsterdamUMCdb.

In [ ]:
#@title Authenticate User { display-mode: "form" }
auth.authenticate_user()
print('Successfully authenticated')

## 3. Initialize BigQuery Client
This cell sets up the client and default configurations for querying.

In [ ]:
#@title Initialize Client { display-mode: "form" }
from google.cloud import bigquery

# Default query job configuration
def_config = bigquery.job.QueryJobConfig(default_dataset=f"{DATASET_PROJECT_ID}.{DATASET_ID}")

# Create the BigQuery client
client = bigquery.Client(project=PROJECT_ID, location=LOCATION, default_query_job_config=def_config)

# Enable data table display in Colab
%load_ext google.colab.data_table
from google.colab.data_table import DataTable
DataTable.max_columns = 50
DataTable.max_rows = 20000

print("BigQuery client initialized and ready.")

# Calculate Detailed Fluid Balance

For testing purposes: Single visit validation

In [ ]:
# @title Single Visit Validator (Manual Check - Calendar Day)
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from google.cloud import bigquery

# --- 1. PARAMETERS ---
# Enter the Visit ID you want to audit (from the main result table)
VISIT_ID = 34050 #@param {type:"integer"}

# Client Setup
client = bigquery.Client(project=PROJECT_ID)
dataset_ref = f"{DATASET_PROJECT_ID}.{DATASET_ID}"
job_config = bigquery.QueryJobConfig(default_dataset=dataset_ref)

# --- 2. CONFIGURATION (Must match main script) ---
INPUT_IDS = [
    3037253, # Intravascular
    3010494, # Enteral
    3006552  # Oral
]

OUTPUT_IDS = [
    3014315, # Urine
    3011087, # Stool
    3026556, # Chest Tube
    3018767, # Ventricle Drain
    21491183, # GI Drain
    3020433  # Misc
]

# Generate SQL lists
input_sql_list = ", ".join(map(str, INPUT_IDS))
output_sql_list = ", ".join(map(str, OUTPUT_IDS))
all_ids_sql_list = ", ".join(map(str, INPUT_IDS + OUTPUT_IDS))

# --- 3. FETCH RAW DATA ---
print(f"Fetching raw fluid measurements for Visit ID: {VISIT_ID}...")

query = f"""
SELECT
    m.measurement_datetime,
    m.measurement_concept_id,
    m.value_as_number
FROM measurement m
JOIN visit_occurrence v ON m.person_id = v.person_id
WHERE v.visit_occurrence_id = {VISIT_ID}
  AND m.measurement_concept_id IN ({all_ids_sql_list})
  -- Get all data strictly within the visit window
  AND m.measurement_datetime BETWEEN v.visit_start_datetime AND IFNULL(v.visit_end_datetime, CURRENT_TIMESTAMP())
ORDER BY m.measurement_datetime
"""

df_raw = client.query(query, job_config=job_config).to_dataframe()

if df_raw.empty:
    print(f"❌ No fluid data found for Visit ID {VISIT_ID}.")
else:
    # --- 4. PYTHON CALCULATION (Calendar Day Logic) ---
    # Convert to datetime (Pandas handles BQ timestamps usually as UTC)
    df_raw['measurement_datetime'] = pd.to_datetime(df_raw['measurement_datetime'])

    # Check if data is timezone aware
    is_tz_aware = df_raw['measurement_datetime'].dt.tz is not None

    # Categorize
    def get_category(cid):
        if cid in INPUT_IDS: return 'Input'
        if cid in OUTPUT_IDS: return 'Output'
        return 'Unknown'

    df_raw['Category'] = df_raw['measurement_concept_id'].apply(get_category)

    # Logic: Simple Calendar Date
    df_raw['Report_Date'] = df_raw['measurement_datetime'].dt.date

    # --- 5. AGGREGATED TABLE ---
    print("\n--- 1. Aggregated Daily Balance (Calendar Day) ---")
    print("Compare this table with your BigQuery SQL results.")

    daily = df_raw.groupby(['Report_Date', 'Category'])['value_as_number'].sum().unstack(fill_value=0)

    # Ensure columns exist
    if 'Input' not in daily: daily['Input'] = 0
    if 'Output' not in daily: daily['Output'] = 0

    daily['Daily_Balance'] = daily['Input'] - daily['Output']
    # Calculate Cumulative Balance
    daily['Cumulative_Balance'] = daily['Daily_Balance'].cumsum()

    display(daily.style.format("{:,.0f}").background_gradient(cmap='RdBu', subset=['Daily_Balance']))

    # --- 6. VISUALIZATION ---
    print("\n--- 2. Visualization: Cumulative Balance over Time ---")

    plt.figure(figsize=(14, 6))

    # Prepare flow for plotting
    df_raw['Flow'] = df_raw.apply(lambda x: x['value_as_number'] if x['Category']=='Input' else -x['value_as_number'], axis=1)
    df_raw['CumSum_Exact'] = df_raw['Flow'].cumsum()

    # Plot Line
    plt.plot(df_raw['measurement_datetime'], df_raw['CumSum_Exact'], label='Cumulative Balance', color='purple', linewidth=2)

    # Mark Midnight (00:00) for every day involved
    days = df_raw['measurement_datetime'].dt.date.unique()

    # Get min timestamp for comparison
    min_ts = df_raw['measurement_datetime'].min()

    for day in days:
        # Create timestamp for midnight
        midnight = pd.Timestamp(day)

        # FIX: Make midnight timezone-aware if the data is timezone-aware
        if is_tz_aware:
            midnight = midnight.tz_localize(df_raw['measurement_datetime'].dt.tz)

        # Only plot if it falls within or after the start of data
        if midnight >= min_ts:
            plt.axvline(midnight, color='gray', linestyle='--', alpha=0.5)
            plt.text(midnight, df_raw['CumSum_Exact'].min(), ' 00:00', fontsize=8, color='gray', rotation=90)

    plt.title(f'Fluid Balance (Calendar Days) for Visit {VISIT_ID}')
    plt.ylabel('Cumulative Volume (ml)')
    plt.xlabel('Time')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%d %H:%M'))
    plt.show()

    # --- 7. RAW DATA SAMPLE ---
    print("\n--- 3. Raw Data Sample ---")
    display(df_raw[['measurement_datetime', 'Category', 'value_as_number', 'Report_Date']].head(5))

# Script for extraction of daily fluid balances

In [ ]:
# @title Fluid Balance Extraction (Daily Aggregation)
import pandas as pd
from google.cloud import bigquery

# --- 1. Client & Dataset Setup ---
# Initialize BigQuery client using project and dataset variables defined globally
client = bigquery.Client(project=PROJECT_ID)
dataset_ref = f"{DATASET_PROJECT_ID}.{DATASET_ID}"
job_config = bigquery.QueryJobConfig(default_dataset=dataset_ref)

print(f"Fluid Balance: Client configured for dataset '{dataset_ref}'")


# Concept ID Configuration
# Define which concepts constitute Input (Intake) vs. Output (Losses)
INPUT_IDS = [
    3037253, # Intravascular
    3010494, # Enteral
    3006552  # Oral
]

OUTPUT_IDS = [
    3014315, # Urine
    3011087, # Stool
    3026556, # Chest Tube
    3018767, # Ventricle Drain
    21491183, # GI Drain
    3020433  # Misc
]

# --- 3. Query Construction ---
# Format lists for SQL IN clauses
input_sql_list = ", ".join(map(str, INPUT_IDS))
output_sql_list = ", ".join(map(str, OUTPUT_IDS))
all_ids_sql_list = ", ".join(map(str, INPUT_IDS + OUTPUT_IDS))

# --- 4. SQL Execution ---
# Logic: Aggregates fluid data by Person, Visit, and Calendar Date.
# Calculates Daily Balance (Input - Output) and Cumulative Balance per Visit.
query_text = f"""
WITH vlist AS (
  -- Identify target cohort (ventilated patients)
  SELECT DISTINCT person_id
  FROM measurement m
  JOIN concept c ON m.measurement_concept_id = c.concept_id
  WHERE concept_name LIKE '%ventila%'
),
labeled_data AS (
  SELECT
    m.person_id,
    v.visit_occurrence_id,
    m.measurement_datetime,
    m.value_as_number,
    -- Extract Calendar Date from timestamp
    DATE(m.measurement_datetime) as report_date,
    -- Classify measurements based on configured IDs
    CASE
      WHEN m.measurement_concept_id IN ({input_sql_list}) THEN 'Input'
      WHEN m.measurement_concept_id IN ({output_sql_list}) THEN 'Output'
      ELSE NULL
    END as category
  FROM measurement m
  JOIN visit_occurrence v
    ON m.person_id = v.person_id
    AND m.measurement_datetime >= v.visit_start_datetime
    AND (v.visit_end_datetime IS NULL OR m.measurement_datetime <= v.visit_end_datetime)
  WHERE m.measurement_concept_id IN ({all_ids_sql_list})
  AND m.person_id IN (SELECT person_id FROM vlist)
),
daily_agg AS (
  SELECT
    person_id,
    visit_occurrence_id,
    report_date,
    SUM(CASE WHEN category = 'Input' THEN value_as_number ELSE 0 END) as total_input,
    SUM(CASE WHEN category = 'Output' THEN value_as_number ELSE 0 END) as total_output
  FROM labeled_data
  GROUP BY person_id, visit_occurrence_id, report_date
)
SELECT
  person_id,
  visit_occurrence_id,
  report_date,
  CAST(total_input AS INT64) as total_input,
  CAST(total_output AS INT64) as total_output,
  CAST((total_input - total_output) AS INT64) as daily_balance,
  -- Calculate Cumulative Sum partitioned by Visit
  CAST(SUM(total_input - total_output) OVER (
    PARTITION BY person_id, visit_occurrence_id
    ORDER BY report_date
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS INT64) as cumulative_balance
FROM daily_agg
ORDER BY person_id, report_date
"""

print("Executing BigQuery extraction...")
fluid_balance = client.query(query_text, job_config=job_config).to_dataframe()

# --- 5. Result Processing ---
if fluid_balance.empty:
    print("WARNING: No data returned for the specified criteria.")
else:
    # Ensure correct datetime format
    fluid_balance['report_date'] = pd.to_datetime(fluid_balance['report_date'])

    print(f"Extraction complete. Retrieved {len(fluid_balance)} records.")
    display(fluid_balance.head())